In [1]:
from numpy import *
from scipy.sparse import *
import implicit

In [30]:
playlists=genfromtxt("train_final.csv",skip_header=1,usecols=0).astype(int32)
playlists

array([ 3271849,  5616275, 11267488, ...,  2659739, 10687820,  4962329], dtype=int32)

In [12]:
tracks=genfromtxt("train_final.csv",skip_header=1,usecols=1).astype(int32)
tracks

array([2801526,  727878, 2805283, ..., 1468602, 1187581, 1094523], dtype=int32)

In [13]:
playlists.shape,tracks.shape

((1040522,), (1040522,))

In [14]:
data=ones(playlists.shape)
data

array([ 1.,  1.,  1., ...,  1.,  1.,  1.])

In [15]:
# MAGIA_1 = creare velocemente la URM
URM=coo_matrix((data,(tracks,playlists)))
URM

<5018275x11766363 sparse matrix of type '<class 'numpy.float64'>'
	with 1040522 stored elements in COOrdinate format>

In [16]:
URM=csc_matrix(URM).astype(int32)

In [17]:
max(tracks)

5018274

In [18]:
max(playlists)

11766362

In [38]:
#faccio la trasposta ma si poteva anche creare direttamente trasposta
URM=URM.T
URM

<11766363x5018275 sparse matrix of type '<class 'numpy.int32'>'
	with 1040522 stored elements in Compressed Sparse Row format>

In [39]:
#P=URM*URM.T
#P

In [40]:
M=URM

In [41]:
# MAGIA_2 = creare le reduced velocemente
M = M[M.getnnz(1)>0][:,M.getnnz(0)>0]
M

<45649x99999 sparse matrix of type '<class 'numpy.int32'>'
	with 1040522 stored elements in Compressed Sparse Row format>

In [42]:
t99999=unique(tracks.astype(int32))

In [43]:
t100k=load("ordered_tracks.npy")

In [44]:
#questa track non compare in nessuna playlist
setdiff1d(t100k,t99999)

array([ 3626362.])

In [45]:
from implicit.als import AlternatingLeastSquares
#from implicit.approximate_als import (AnnoyAlternatingLeastSquares, NMSLibAlternatingLeastSquares,FaissAlternatingLeastSquares)
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,
                                         TFIDFRecommender, bm25_weight)

In [46]:
MODELS = {"als":  AlternatingLeastSquares,
          "tfidf": TFIDFRecommender,
          "cosine": CosineRecommender,
          "bm25": BM25Recommender}

In [47]:
def get_model(model_name):
    model_class = MODELS.get(model_name)
    if not model_class:
        raise ValueError("Unknown Model '%s'" % model_name)

    # some default params
    if issubclass(model_class, AlternatingLeastSquares):
        params = {'factors': 50, 'dtype': numpy.float32}
    elif model_name == "bm25":
        params = {'K1': 100, 'B': 0.5}
    else:
        params = {}

    return model_class(**params)

In [48]:
modelCos=get_model("cosine")
modelTFIDF=get_model("tfidf")

In [49]:
modelCos.fit(M)
modelTFIDF.fit(M)

In [50]:
simCos=modelCos.similarity
simTFIDF=modelTFIDF.similarity

In [51]:
simCos.setdiag(0)
simTFIDF.setdiag(0)
simCos,simTFIDF

(<45649x45649 sparse matrix of type '<class 'numpy.float64'>'
 	with 861653 stored elements in Compressed Sparse Row format>,
 <45649x45649 sparse matrix of type '<class 'numpy.float64'>'
 	with 861653 stored elements in Compressed Sparse Row format>)

In [52]:
#cosine
argsort(simCos[0].todense())

matrix([[    0, 30423, 30424, ...,  4796, 44772, 19688]])

In [53]:
#tfidf
argsort(simTFIDF[0].todense())

matrix([[    0, 30422, 30423, ..., 38467, 44772, 19688]])

In [82]:
simSum=simTFIDF+simCos
simSum

<45649x45649 sparse matrix of type '<class 'numpy.float64'>'
	with 975058 stored elements in Compressed Sparse Row format>

In [83]:
argsort(simSum[0].todense())

matrix([[    0, 30422, 30423, ...,  4796, 44772, 19688]])

In [84]:
simSum

<45649x45649 sparse matrix of type '<class 'numpy.float64'>'
	with 975058 stored elements in Compressed Sparse Row format>

In [124]:
targetPl=genfromtxt("target_playlists.csv",skip_header=1,dtype=int32)
targetPl

array([10024884, 10624787,  4891851, ..., 11369546,  7939535,   297021], dtype=int32)

In [31]:
uniquePl=unique(playlists)
uniquePl.size

45649

In [125]:
targetMask=array(list(map(lambda x: isin(x,targetPl),uniquePl)))
targetMask.size

45649

In [3]:
#targetSimSum=simSum[targetMask]
#save_npz("simSum_TargetPl_AllPl",targetSimSum)
targetSimSum=load_npz("simSum_TargetPl_AllPl.npz")
targetSimSum

<10000x45649 sparse matrix of type '<class 'numpy.float64'>'
	with 235509 stored elements in Compressed Sparse Row format>

In [127]:
targetStrangeOrder=uniquePl[targetMask]
targetStrangeOrder.size

10000

In [12]:
pIdForTracks=load("playlists_with_tracks.npy")[:,0]
pIdForTracks#sono solo 10k

array([10024884, 10624787,  4891851, ..., 11369546,  7939535,   297021])

In [7]:
pTracks=load("playlists_with_tracks.npy")[:,1:]
pTracks#sono solo 10k

array([[2879006, 1532328, 3027673, ...,       0,       0,       0],
       [2090319, 2152295, 3230559, ...,       0,       0,       0],
       [2621486, 2895498, 1369381, ...,       0,       0,       0],
       ..., 
       [ 416692, 1587449, 1057389, ...,       0,       0,       0],
       [2788577, 3090083, 1609308, ...,       0,       0,       0],
       [2789282, 2154608,  973448, ...,       0,       0,       0]])

In [55]:
allPlIdWithTracks=load_npz("all_playlists_with_tracks.npz")
allPlIDs=squeeze(asarray(allPlIdWithTracks[:,0].todense()))
allPlTracks=allPlIdWithTracks[:,1:]

In [22]:
monoArtist=load("target_playlist_mono_artist.npy")
monoArtistPlID=monoArtist[:,0]
monoArtistArtistID=monoArtist[:,1]
artistIndexes=load("uniqueArtists_NeededToIndexThe_ArtistReducedMatrices.npy")
artistTracks=load_npz("artists_with_tracksID_ordered_by_occurrencies.npz")
target_tracks_ordered = load("targetTracksOrdered.npy")

In [70]:
def tracksForMonoArtist(pID,tracksOfThisPl):
    artist=monoArtistArtistID[where(monoArtistPlID==pID)[0][0]]
    artistIndex=where(artistIndexes==artist)[0][0]
    thisArtistTracks=artistTracks.getrow(artistIndex).data
    return array([t for t in thisArtistTracks if not isin(t,tracksOfThisPl) and isin(t,target_tracks_ordered)])

In [121]:
def tracksForGeneric(p,tracksOfThisPl):
    plIndexes=asarray(flip(argsort(p.todense()),1)).flatten()[0:10]
    similarPlIDs=uniquePl[plIndexes]
    candidateTracks=[]
    for similarPlID in similarPlIDs:
        indexToFindTracksOfThisPl=where(allPlIDs==similarPlID)[0][0]
        tracksOfThisPl=allPlTracks[indexToFindTracksOfThisPl].data
        candidateTracks+=list(tracksOfThisPl)
    myArr= array([t for t in array(candidateTracks) if ((not isin(t,tracksOfThisPl)) and isin(t,target_tracks_ordered))])
    return myArr

In [79]:
def callIfNotEnough(n,tracksOfThisPl,suggestions):
    newSuggestions=[]
    for t in target_tracks_ordered:
        if(len(newSuggestions)==n):
            break
        elif(not isin(t,tracksOfThisPl) and not isin(t, suggestions)):
            newSuggestions.append(t)
    return concatenate((suggestions,array(newSuggestions)))

In [128]:
with open("newSimTestMonoArtist.csv","w") as f:
    f.write("playlist_id,track_ids\n")
    for p,pID in zip(targetSimSum,targetStrangeOrder):
        f.write(str(pID)+",")
        indexToFindTracksOfThisPl=where(allPlIDs==pID)[0][0]
        tracksOfThisPl=allPlTracks[indexToFindTracksOfThisPl].data
        if (isin(pID,monoArtistPlID)):
            suggestions=tracksForMonoArtist(pID,tracksOfThisPl)
        else:
            suggestions=tracksForGeneric(p,tracksOfThisPl)
        if(suggestions.size>=5):
            suggestions=suggestions.take(range(5))
        else:
            n=5-suggestions.size
            suggestions=callIfNotEnough(n,tracksOfThisPl,suggestions)
        for t in suggestions:
            f.write(str(int(t))+" ")
        f.write("\n")

shit
shit
shit
shit
shit
shit
shit
shit
shit
shit
shit
shit
shit
shit
shit
shit
shit
shit
shit
shit
shit
shit
shit
shit
shit


array([1, 2, 3, 4])